In [12]:
import pandas as pd
import numpy as np

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM,Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [14]:
from datasets import Dataset

In [19]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    LlamaTokenizer, 
    LlamaForCausalLM,
)

from peft import LoraConfig, PeftModel,TaskType, get_peft_model
#from bitsandbytes import quantize_model
from trl import SFTTrainer

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


ImportError: cannot import name 'quantize_model' from 'bitsandbytes' (E:\modelo\pythonProject\venv\Lib\site-packages\bitsandbytes\__init__.py)

In [20]:
print(torch.version.cuda)

None


In [21]:
print(torch.cuda.is_available())
print(torch.version.cuda)

False
None


In [22]:
df = pd.read_csv('preguntas_respuesta')

In [23]:

df['text'] = df['pregunta'] + df['respuesta']

In [24]:
df = df[0:10]

In [25]:
df=df.reset_index()

In [26]:
nDf= df.loc[:,'text']

In [27]:
dataset = Dataset.from_pandas(pd.DataFrame({'text': nDf}))

In [28]:
def tokenize_function(examples):
    outputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs


In [34]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [30]:
model_name = "acalatrava/TinyLlama-1.1B-translate-en-es"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [18]:
#model = quantize_model(model, bits=4)

NameError: name 'quantize_model' is not defined

In [ ]:
'''lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4,  # rango de la factorización
    lora_alpha=32,  # multiplicador de la matriz de baja dimensión
    lora_dropout=0.1,  # probabilidad de dropout
)'''

In [ ]:
#model = get_peft_model(model, lora_config)

In [35]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # No queremos enmascarado de lenguaje para GPT-2
)
training_args = TrainingArguments(
    output_dir="./results",  # Directorio donde se guardarán los resultados
    overwrite_output_dir=True,
    num_train_epochs=1,  # Número de épocas de entrenamiento
    per_device_train_batch_size=2,  # Tamaño del lote de entrenamiento por dispositivo
    save_steps=1,  # Pasos después de los cuales guardar el modelo
    save_total_limit=2,
    weight_decay=0.01,
    remove_unused_columns=False
)

# Definir el objeto Trainer y entrenar el modelo
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator
)


In [36]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=5, training_loss=3.9157638549804688, metrics={'train_runtime': 1339.1379, 'train_samples_per_second': 0.007, 'train_steps_per_second': 0.004, 'total_flos': 7945055109120.0, 'train_loss': 3.9157638549804688, 'epoch': 1.0})

In [38]:
trainer.save_model('llama')